# Tokenizer Dev

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.insert(0, "/code/huggingface/transformers-fair-wmt/src")

In [3]:
import torch
from pprint import pprint

In [4]:
sentence = "Machine Learning is great"
sentence_0 = "Machine Learning is great"
sentence_1 = "That's right"

# Baseline

In [5]:
#checkpoint_file='model1.pt:model2.pt:model3.pt:model4.pt'
checkpoint_file='model4.pt'
model = torch.hub.load('pytorch/fairseq', 'transformer.wmt19.en-ru', checkpoint_file=checkpoint_file, tokenizer='moses', bpe='fastbpe')

Using cache found in /home/stas/.cache/torch/hub/pytorch_fairseq_master


In [14]:
# encode step by step
tokens = model.tokenize(sentence)
"tokenize ", len(tokens), "-".join(tokens)
bpe = model.apply_bpe(tokens)
bpe_str = str(bpe).split()
"apply_bpe: ", len(bpe_str), bpe_str
bin = model.binarize(bpe)
"binarize: ", len(bin), bin

# compare to model.encode - should give us the same output
expected = model.encode(sentence)
"encode:   ", len(expected), expected

('tokenize ', 25, 'M-a-c-h-i-n-e- -L-e-a-r-n-i-n-g- -i-s- -g-r-e-a-t')

('apply_bpe: ', 6, ['Mach@@', 'ine', 'Lear@@', 'ning', 'is', 'great'])

('binarize: ', 7, tensor([10217,  1419,     3,  2515,    21,  1054,     2]))

('encode:   ', 7, tensor([10217,  1419,     3,  2515,    21,  1054,     2]))

# Target

In [15]:
from transformers.tokenization_fsmt import FSMTTokenizer

In [17]:
tokenizer = FSMTTokenizer.from_pretrained('facebook/wmt19-en-ru')

In [18]:
tokens = tokenizer.tokenize(sentence)
input_ids = tokenizer(sentence)['input_ids']
len(tokens), tokens
len(input_ids), input_ids
#tokenizer._convert_id_to_token(31232)

(6, ['Mach', 'ine</w>', 'Lear', 'ning</w>', 'is</w>', 'great</w>'])

(7, [10217, 1419, 3, 2515, 21, 1054, 2])

In [19]:
from numpy.testing import assert_array_equal
assert_array_equal(expected.numpy(), input_ids, "did we get it right?")

# 2 sequences

In [20]:
enc = tokenizer.encode_plus(sentence_0, sentence_1, add_special_tokens=True)
pprint(enc)
for k in enc.keys(): print(f"{k}: {len(enc[k])}")


{'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
 'input_ids': [10217, 1419, 3, 2515, 21, 1054, 2, 1137, 39, 231, 2],
 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1]}
input_ids: 11
token_type_ids: 11
attention_mask: 11


In [21]:
"Success!!!"

'Success!!!'